<a href="https://colab.research.google.com/github/Anshita5/exact_interence_unit5/blob/main/exact_inference05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install necessary library
!pip install pgmpy

# Importing necessary libraries with different aliases
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder as LE
from pgmpy.models import BayesianNetwork as BN
from pgmpy.factors.discrete import TabularCPD as CPD
from pgmpy.inference import VariableElimination as VE

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.7 MB/s eta 0:00:00


In [4]:
# Load dataset
data_frame = pd.read_csv("/content/dataset.csv")

In [5]:
# Define a function for encoding labels
def encode_labels(data_frame):
    encoder = LE()  # Use the aliased LabelEncoder
    for column in data_frame.columns:
        data_frame[column] = encoder.fit_transform(data_frame[column])
    return data_frame

In [6]:
# Preprocess the data
data_encoded = encode_labels(data_frame)


In [7]:
# Create the Bayesian Network model
def build_bayesian_network():
    # Define the model structure
    bayesian_model = BN([('Symptom1', 'Disease'),
                         ('Symptom2', 'Disease'),
                         ('Symptom3', 'Disease')])

    # Define the CPDs (Conditional Probability Distributions)
    cpd_symptom1 = CPD(variable='Symptom1', variable_card=2, values=[[0.8], [0.2]])
    cpd_symptom2 = CPD(variable='Symptom2', variable_card=2, values=[[0.7], [0.3]])
    cpd_symptom3 = CPD(variable='Symptom3', variable_card=2, values=[[0.6], [0.4]])
    cpd_disease = CPD(variable='Disease', variable_card=2, values=[[0.9, 0.4, 0.2],
                                                                    [0.1, 0.6, 0.8]],
                      evidence=['Symptom1', 'Symptom2', 'Symptom3'], evidence_card=[2, 2, 2])

    # Add the CPDs to the model
    bayesian_model.add_cpds(cpd_symptom1, cpd_symptom2, cpd_symptom3, cpd_disease)

    # Check if the model is valid
    assert bayesian_model.check_model(), "The Bayesian network model is invalid!"

    return bayesian_model

In [9]:
# Create the Bayesian Network model
def build_bayesian_network():
    # Define the model structure
    bayesian_model = BN([('Symptom1', 'Disease'),
                         ('Symptom2', 'Disease'),
                         ('Symptom3', 'Disease')])

    # Define the CPDs (Conditional Probability Distributions)
    cpd_symptom1 = CPD(variable='Symptom1', variable_card=2, values=[[0.8], [0.2]])
    cpd_symptom2 = CPD(variable='Symptom2', variable_card=2, values=[[0.7], [0.3]])
    cpd_symptom3 = CPD(variable='Symptom3', variable_card=2, values=[[0.6], [0.4]])

    # Updated CPD for Disease (now has 8 columns for all combinations of evidence)
    cpd_disease = CPD(variable='Disease', variable_card=2, values=[[0.9, 0.7, 0.6, 0.4, 0.3, 0.2, 0.1, 0.05],
                                                                    [0.1, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9, 0.95]],
                      evidence=['Symptom1', 'Symptom2', 'Symptom3'], evidence_card=[2, 2, 2])

    # Add the CPDs to the model
    bayesian_model.add_cpds(cpd_symptom1, cpd_symptom2, cpd_symptom3, cpd_disease)

    # Check if the model is valid
    assert bayesian_model.check_model(), "The Bayesian network model is invalid!"

    return bayesian_model

# Build the network
network_model = build_bayesian_network()



In [11]:
# Perform inference using Variable Elimination
def infer_disease(bayesian_model, symptoms_evidence):
    infer = VE(bayesian_model)  # Using the aliased VariableElimination
    # Query the probability of having a disease given the symptoms
    query_result = infer.query(variables=['Disease'], evidence=symptoms_evidence)
    return query_result

# Example of inference with hypothetical symptoms
evidence_example = {'Symptom1': 1, 'Symptom2': 0, 'Symptom3': 1}
disease_prediction = infer_disease(network_model, evidence_example)
print("Predicted Disease:", disease_prediction)


Predicted Disease: +------------+----------------+
| Disease    |   phi(Disease) |
+============+================+
| Disease(0) |         0.2000 |
+------------+----------------+
| Disease(1) |         0.8000 |
+------------+----------------+
